In [30]:
import pandas as pd
import re

In [31]:
# Provided data: List of MT103 messages
mt103_messages = [
    """
    {1:F01MYMBGB2L0XXX0000000000}{2:I103HBUKGB4BXXXN}{3:{108:MT103
    0001}}{4:
    :20:MT103 0001
    :23B:CRED
    :32A:210322AED5000,
    :50K:/DE98765432101234567890
    COMMERZBANK AG
    HAMBURG, GERMANY
    /COBADEHHXXX
    :52A:/COBADEHHXXX
    COMMERZBANK AG
    HAMBURG, GERMANY
    :53A:/MYMBGB2LXXX
    METRO BANK PLC
    LONDON, UNITED KINGDOM
    :57A:/HBUKGB4BXXX
    HSBC BANK PLC
    LONDON, UNITED KINGDOM
    :59:/GB57METR12345678901234
    NORDFISCH GMBH
    BODENSEE STR. 226
    22761 HAMBURG
    GERMANY
    :71A:OUR
    :71F:/BIC/HBUKGB4BXXX
    :71G:/INS/THIS IS A PAYMENT FOR TUNA SUPPLY
    -}
    """,
    # Add more MT103 messages here
    """"
    {1:F01ABCBUS33AXXX0000000000}{2:I103HSBCHKHHHKXXXN}{3:{108:MT103
    0001}}{4:
    :20:MT103 0001
    :23B:CRED
    :32A:210322USD10000,
    :50K:/US12345678901234567890
    ABC INDUSTRIES
    123 MAIN STREET
    NEW YORK, NY 10001
    UNITED STATES
    /ABCBUS33XXX
    :52A:/ABCBUS33XXX
    ABC BANK
    NEW YORK, NY
    UNITED STATES
    :53A:/HSBCHKHHHKXXX
    HSBC HONG KONG
    1 QUEEN'S ROAD CENTRAL
    Unset
    HONG KONG
    :54A:/CITIHKHX
    CITIBANK HONG KONG
    3 GARDEN ROAD
    CENTRAL, HONG KONG
    :56A:/ICBKCNBJGZU
    INDUSTRIAL AND COMMERCIAL BANK OF CHINA
    GUANGZHOU BRANCH
    76 HUANSHI ROAD WEST
    GUANGZHOU, CHINA
    :57A:/CITIUS33
    CITIBANK NA
    111 WALL STREET
    NEW YORK, NY 10043
    UNITED STATES
    :59:/CN123456789012345678
    XYZ SUPPLIERS
    123 HUANGPU ROAD
    SHANGHAI, CHINA
    :71A:OUR
    :71F:/BIC/HSBCHKHHHKXXX
    :71G:/MSG/PAYMENT FOR GOODS
    -}""",

    """"
    {1:F01ABNANL2AXXX0000000000}{2:I103SCBLGB2LXXXXN}{3:{103:TGT}{108
    :MT103 0001}}{4:
    :20:MT103 0001
    :23B:CRED
    :32A:210322USD9899,
    :50A:/NL20ABNA0404875234
    ABNANL2A
    ABC SUPPLIERS BV
    AMSTERDAM, NETHERLANDS
    :56A:/SCBLGB2LXXX
    STANDARD CHARTERED BANK
    LONDON, UK
    :57A:/BNYMUS33XXX
    BNY MELLON
    NEW YORK, NY, US
    :59:/PASSNGLAXXX
    AFRICAN EXPORT-IMPORT BANK
    LAGOS, NIGERIA
    XYZ ENTERPRISES LTD
    LAGOS, NIGERIA
    :70:INV NO. 12345
    REF. 98765
    SUPPLY OF GOODS AS PER PURCHASE ORDER NO. 54321
    -}"""
    ]

In [32]:
def extract_value(tag, message):
    regex_pattern = r'(?<=:' + tag + r':)(.*?)(?=:\d{2}[A-Z]|\Z)'
    match = re.search(regex_pattern, message, re.DOTALL)  # re.DOTALL zorgt ervoor dat '.' ook newlines matcht
    if match:
        return match.group(1).strip()
    else:
        return ''

In [33]:
# Create an empty DataFrame with columns as per the provided schema
columns = [
    "transaction_date", 
    "transaction_id",
    "transaction_message", 
    "transaction_currency",
    "transaction_amount",
    "transaction_type", 
    "transaction_direction",
    "transaction_status",
    "instrument_type", 
    "originator_full_name",
    "originator_first_name",
    "originator_account_number",
    "originator_middle_names_patronymic",
    "originator_last_name", 
    "originator_address",
    "originator_country",
    "originator_account_number",
    "originator_branch_id",
    "originator_bic",
    "originator_fi_name",
    "originator_fi_country",
    "incoming_intermediary_fi_bic",
    "outgoing_intermediary_fi_bic",
    "beneficiary_full_name",
    "beneficiary_first_name",
    "beneficiary_middle_name_patronymic",
    "beneficiary_last_name",
    "beneficiary_address",
    "beneficiary_country",
    "beneficiary_account_number",
    "beneficiary_branch_id",
    "beneficiary_bic",
    "beneficiary_fi_name",
    "beneficiary_fi_country",     
]

empty_df = pd.DataFrame(columns=columns)

In [34]:

def determine_transaction_direction(mt103_message, known_originator, known_beneficiary):

    originator_field = extract_value('50K', mt103_message)
    beneficiary_field = extract_value('59', mt103_message)
    
    if known_originator.lower() in originator_field.lower():
        return "o"  # Uitgaand
    elif known_beneficiary.lower() in beneficiary_field.lower():
        return "i"  # Inkomend
    else:
        return "io"  # Onbepaald
    
    # Voorbeeldgebruik:
known_originator = "Industries"
known_beneficiary = "Suppliers"

In [35]:
import pandas as pd

# Assuming mt103_messages is a list of MT103 message strings and extract_value is a predefined function
dfs = []
for mt103_message in mt103_messages:
    
    # Extract values from the MT103 message
    transaction_id = extract_value('20', mt103_message)
    transaction_message = extract_value('71G', mt103_message)

    value_field = extract_value('32A', mt103_message)
    transaction_currency_match = re.match(r'\d{6}([A-Z]{3})', value_field)
    transaction_currency = transaction_currency_match.group(1) if transaction_currency_match else None
    transaction_amount = re.sub(r'^\d{6}[A-Z]{3}', '', value_field).replace(',', '') if transaction_currency else None

    transaction_type = extract_value('23B', mt103_message)

    transaction_direction = determine_transaction_direction(mt103_message, known_originator, known_beneficiary)
    # Update de data dictionary met de nieuwe transaction_direction waarde
    # data["transaction_direction"] = transaction_direction

    transaction_status = None

    instrument_type = None

    originator_field_50K = extract_value('50K', mt103_message)
    
    if originator_field_50K:
        # Gebruik jouw bestaande logica voor 50K
        # Bijvoorbeeld:
        originator_components = originator_field_50K.split('\n')
        # Verder verwerken zoals je al hebt gedaan...
        if len(originator_components) >= 4:
            originator_account_number = originator_components[0].strip()
            originator_full_name = originator_components[1].strip()
            originator_address = originator_components[2].strip()
            originator_Country_bic = originator_components[3].strip()

        if len(originator_components) >= 3:
            # Assume the first line is the account number, the second line is the full name
            originator_account_number = originator_components[0].strip()
            originator_full_name = originator_components[1].strip()
            
            # Now let's dynamically extract the address
            # The address starts after the name and continues until we hit a line that is likely a country or BIC
            address_components = []
            for component in originator_components[2:]:
                # Break if we encounter a country or BIC line
                if re.match(r"^[A-Z\s]+(?:[A-Z]{2})?$", component.strip()) or '/' in component:
                    break
                address_components.append(component.strip())
            originator_address = ', '.join(address_components)
        
            # Aanname: De volledige naam kan gesplitst worden in voornaam, middelste namen, en achternaam.
            name_parts = originator_full_name.split()
            if len(name_parts) > 2:  # Aanname: Er zijn middelste namen of een patroniem.
                originator_first_name = name_parts[0]
                originator_last_name = name_parts[-1]
                originator_middle_names_patronymic = ' '.join(name_parts[1:-1])
            elif len(name_parts) == 2:  # Alleen voornaam en achternaam.
                originator_first_name = name_parts[0]
                originator_middle_names_patronymic = ''
                originator_last_name = name_parts[1]
            else:  # Alleen één naam beschikbaar.
                originator_first_name = name_parts[0]
                originator_middle_names_patronymic = ''
                originator_last_name = ''

    else: 
        # Als 50K leeg is, probeer 50A
        originator_field_50A = extract_value('50A', mt103_message)
        if originator_field_50A:
            # Verwerk de 50A-specifieke logica hier
            # Dit hangt af van hoe je de gegevens van 50A wilt verwerke
            # Voorbeeld van eenvoudige splitsing, aanpassen op basis van de verwachte structuur
            originator_components_50A = originator_field_50A.split('\n')
            # Mogelijk andere verwerking nodig afhankelijk van je behoeften

    #originator_fi_name
    fi_name_matches = re.findall(r':52A:/(.+?)\n(.+?)\n', mt103_message)
    if fi_name_matches:
        originator_fi_name = fi_name_matches[0][1].strip()
    else:
        fi_name_matches = re.findall(r':50A:/(.+?)\n(.+?)\n', mt103_message)
        if fi_name_matches:
            originator_fi_name = fi_name_matches[0][1].strip()
        else:
            originator_fi_name = None  # or some default value    
    
    beneficiary_field = extract_value('59a', mt103_message)

    # Split the originator field into its components based on newlines and double slashes
    beneficiary_components = beneficiary_field.split('\n')
    
    if len(beneficiary_components) >= 4:
        beneficiary_account_number = beneficiary_components[0].strip()
        beneficiary_full_name = beneficiary_components[1].strip()
        beneficiary_address = beneficiary_components[2].strip()
        beneficiary_Country_bic = beneficiary_components[3].strip()
     


    # Extract and format the transaction date from the :32A: field
    date_str = extract_value('32A', mt103_message)[:6]  # Assuming the date is always at the start
    transaction_year = int(date_str[:2]) + 2000  # Adjust the century as needed
    transaction_month = int(date_str[2:4])
    transaction_day = int(date_str[4:6])
    transaction_date = f"{transaction_day:02d}-{transaction_month:02d}-{transaction_year}"

    # Create a dictionary with extracted values
    data = {
    "transaction_date": transaction_date,
    "transaction_id": transaction_id,
    "transaction_message": transaction_message,
    "transaction_currency": transaction_currency,
    "transaction_amount": transaction_amount,
    "transaction_type": transaction_type,
    "transaction_direction": transaction_direction,  
    "transaction_status": transaction_status,  
    "instrument_type": instrument_type,  
    
    "originator_full_name": originator_full_name,
    "originator_first_name": originator_first_name,  
    "originator_middle_names_patronymic": originator_middle_names_patronymic,  
    "originator_last_name": originator_last_name, 
    "originator_address": originator_address,
    # "originator_country": originator_country,
    "originator_account_number": originator_account_number,
    # "originator_branch_id": riginator_branch_id,  
    # "originator_bic": originator_bic,
    "originator_fi_name": originator_fi_name,
    # "originator_fi_country": originator_fi_country,  
    # "incoming_intermediary_fi_bic": incoming_intermediary_fi_bic,  
    # "outgoing_intermediary_fi_bic": outgoing_intermediary_fi_bic,  
    "beneficiary_full_name": beneficiary_full_name,  
    # "beneficiary_first_name": beneficiary_first_name,  
    # "beneficiary_middle_name_patronymic": beneficiary_middle_name_patronymic,  
    # "beneficiary_last_name": beneficiary_last_name,  
    "beneficiary_address": beneficiary_address,  
    # "beneficiary_country": beneficiary_country,  
    "beneficiary_account_number": beneficiary_account_number,
    # "beneficiary_branch_id": beneficiary_branch_id,  
    # "beneficiary_bic": beneficiary_bic,
    # "beneficiary_fi_name": beneficiary_fi_name,  
    # "beneficiary_fi_country": beneficiary_fi_country, 
    }
    
    # Create DataFrame from dictionary
    df = pd.DataFrame([data])  # Simplified DataFrame creation
    dfs.append(df)

# Concatenate all DataFrames
result_df = pd.concat(dfs, ignore_index=True)


NameError: name 'beneficiary_full_name' is not defined

In [ ]:
result_df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_account_number,originator_fi_name
0,22-03-2021,MT103 0001,/INS/THIS IS A PAYMENT FOR TUNA SUPPLY\n -},AED,5000,CRED,io,None,None,COMMERZBANK AG,COMMERZBANK,,AG,"HAMBURG, GERMANY",/DE98765432101234567890,COMMERZBANK AG
1,22-03-2021,MT103 0001,/MSG/PAYMENT FOR GOODS\n -},USD,10000,CRED,o,None,None,ABC INDUSTRIES,ABC,,INDUSTRIES,"123 MAIN STREET, NEW YORK, NY 10001",/US12345678901234567890,ABC BANK
2,22-03-2021,MT103 0001,,USD,9899,CRED,io,None,None,ABC INDUSTRIES,ABC,,INDUSTRIES,"123 MAIN STREET, NEW YORK, NY 10001",/US12345678901234567890,ABNANL2A
